In [1]:
# 네이버 뉴스 자율주행 크롤링

In [11]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

In [12]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace('','_')
date = date.replace(':','시') + '분'

In [13]:
# 인풋 생성
query = input('검색 키워드를 입력하세요 : ')
query = query.replace(' ', '+')

news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))

검색 키워드를 입력하세요 : 자율주행
총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : 500


In [14]:
# 키워드 = '자율주행'
# = f'https://m.search.naver.com/search.naver?where=m_news&sm=mtb_jum&query={키워드}'
# print(news_url)
#https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query=%EC%9E%90%EC%9C%A8%EC%A3%BC%ED%96%89


In [15]:
news_url = 'https://search.naver.com/search.naver?where=m_news&sm=mtb_jum&query={}'

req = requests.get(news_url.format(query)) # html 코드 받아오기 <Response [200]>
soup = BeautifulSoup(req.text, 'html.parser') # 받은 코드로 파싱 

In [ ]:
# soup

In [19]:
# 원하는 정보를 담을 변수 생성(딕셔너리) : key 번호, value 뉴스기사정보
news_dict = {}
idx = 0 # 현재 뉴스 번호 
cur_page =1 # 네이버 뉴스 웹 페이지 

In [20]:
print()
print('크롤링 중...')

while idx < news_num:
### 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126) ###
    
    table = soup.find_all('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
    
    for n in a_list[:min(len(a_list), news_num-idx)]:
        news_dict[idx] = {'title' : n.get('title'),
                          'url' : n.get('href') }
        idx += 1

    cur_page += 1
    
    pages = soup.find('div', {'class' : 'sc_page_inner'})
    next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')
    
    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')


크롤링 중...


AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [21]:
! pip install openpyxl

In [22]:
print('크롤링 완료')
print('데이터프레임 변환')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
csv_file_name = '네이버뉴스_{}_{}.csv'.format(query, date)

news_df.to_csv(csv_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}'.format(folder_path, csv_file_name))




크롤링 완료
데이터프레임 변환
엑셀 저장 완료 | 경로 : /Users/minjaering/Desktop/2차project\네이버뉴스_자율주행__2_0_2_2_-_0_6_-_2_0_ _1_1_시_2_4_분.csv
